In [1]:
import os
import sys
import gc
import ctypes
import numpy as np
import pandas as pd
import platform
import logging
from datetime import datetime, timedelta
from pathlib import Path
import shutil
import zipfile
import duckdb 
import warnings
import fastparquet
from tqdm import tqdm 
from typing import List, Optional, Union
import psutil
import time # For timing the execution

warnings.filterwarnings('ignore')


# Optional: adjust pandas display for debugging; you can comment these out
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
!pwd

/Users/kenobi/Documents/naarni/repo/explore_data/5.CanDataAnalysis/ver3


In [3]:
EXTRACTION_DIR = Path("../../../data_points/extracted_parts/261125_2days") 
# CORE_COLS = ["id", ]

In [4]:
def free_mem():
    """Try to return freed memory back to the OS (no-op on some platforms)."""
    try:
        libc = ctypes.CDLL(None)
        if hasattr(libc, "malloc_trim"):
            libc.malloc_trim(0)
    except Exception:
        pass

In [5]:
def drop_msb_and_convert(numbers_list):
    translated_numbers = []
    for number in numbers_list:
        binary_str = format(number, 'b')
        dropped_msb_str = binary_str[1:]
        if not dropped_msb_str:
            new_int_value = 0
        else:
            new_int_value = int(dropped_msb_str, 2)
        translated_numbers.append(new_int_value)
    return translated_numbers


In [6]:
import re

dbc_lines = open("canId_conversions.txt").read().splitlines()

temp_map = {}   # dbc_id → list of 1–108
volt_map = {}   # dbc_id → list of 1–576

current_dbc_id = None
current_type = None  # "temp" or "volt"

temp_pattern = re.compile(r"SG_\s+Pack_Temperature(\d+)")
volt_pattern = re.compile(r"SG_\s+Pack_cellVoltage_(\d+)")

for line in dbc_lines:
    line = line.strip()

    # Detect BO_ lines
    if line.startswith("BO_ "):
        parts = line.split()
        current_dbc_id = int(parts[1])
        name = parts[2]

        if name.startswith("Pack_Temperature"):
            current_type = "temp"
            temp_map[current_dbc_id] = []

        elif name.startswith("Pack_cellVoltage"):
            current_type = "volt"
            volt_map[current_dbc_id] = []

        else:
            current_type = None  # ignore unrelated frames

        continue

    # Inside a BO_ block → extract signal names
    if current_type == "temp":
        m = temp_pattern.search(line)
        if m:
            idx = int(m.group(1))
            temp_map[current_dbc_id].append(idx)

    elif current_type == "volt":
        m = volt_pattern.search(line)
        if m:
            idx = int(m.group(1))
            volt_map[current_dbc_id].append(idx)


In [7]:
print("Temperature blocks:", len(temp_map))
print("Total temperature indices:", sum(len(v) for v in temp_map.values()))

print("Voltage blocks:", len(volt_map))
print("Total voltage indices:", sum(len(v) for v in volt_map.values()))


Temperature blocks: 15
Total temperature indices: 108
Voltage blocks: 145
Total voltage indices: 576


In [8]:
# --- DUCKDB CHUNK GENERATOR (Fixed) ---

def duckdb_chunk_generator(conn, sql_query, chunk_size):
    """Generates Pandas DataFrames in chunks directly from DuckDB cursor."""
    cursor = conn.cursor() 
    cursor.execute(sql_query)
    
    while True:
        # Uses the corrected method name: fetch_df_chunk
        chunk = cursor.fetch_df_chunk(chunk_size) 
        if chunk is None or chunk.empty:
            break
        yield chunk

# --- ROBUST FILE EXTRACTION (Fixed from OSErrors) ---

def extract_files_to_disk(zip_path, output_dir):
    """Cleans directory and extracts all Parquet files from ZIP."""
    if output_dir.exists():
        logging.info(f"🧹 Clearing existing directory: {output_dir.resolve()}")
        # Robust cleanup to avoid OS/lock issues
        try:
            shutil.rmtree(output_dir)
        except OSError:
             for item in output_dir.iterdir():
                if item.is_dir():
                    shutil.rmtree(item)
                else:
                    os.remove(item) 
             os.rmdir(output_dir)

    output_dir.mkdir(parents=True)
        
    logging.info("🔄 Extracting ALL Parquet files from ZIP to disk...")
    try:
        with zipfile.ZipFile(zip_path, "r") as z:
            all_files_to_extract = [f for f in z.namelist() if f.endswith(".parquet")]
            logging.info(f"🔎 Found {len(all_files_to_extract)} total Parquet files in archive.")
            for filename in all_files_to_extract:
                z.extract(filename, path=output_dir)
            return len(all_files_to_extract)
    except FileNotFoundError:
        raise FileNotFoundError(f"❌ ZIP file not found at: {zip_path}") from None

def setup_duckdb_query(
    output_dir,
    utc_start,
    utc_end,
    core_cols=None,
    ids=None,   # NEW
):
    """
    Sets up DuckDB connection and SQL query for reading Parquet files
    with timestamp predicate-pushdown filtering.

    Behavior:
      - If core_cols is None -> SELECT * (all columns)
      - If core_cols is provided -> SELECT only those columns
      - If ids (list of strings) is provided -> adds id IN (...) filter
    """

    parquet_glob_path = str(output_dir.joinpath("**/*.parquet"))

    # SELECT clause
    if core_cols is None:
        select_clause = "SELECT *"
    else:
        col_list = ", ".join([f'"{c}"' for c in core_cols])
        select_clause = f"SELECT {col_list}"

    # ID filter clause
    id_clause = ""
    if ids:
        # Format like: '6','16','28'
        id_list = ", ".join([f"'{x}'" for x in ids])
        id_clause = f"AND id IN ({id_list})"

    # Full SQL
    sql_query = f"""
        {select_clause}
        FROM read_parquet('{parquet_glob_path}')
        WHERE 
            "timestamp" >= '{utc_start.isoformat()}' AND
            "timestamp" <  '{utc_end.isoformat()}'
            {id_clause}
    """

    return duckdb.connect(), sql_query


In [9]:
ids = ['28']


ist_start_str = "2025-11-27 13:00:00.000"
ist_end_str = "2025-11-27 13:10:00.000"

# Parse IST timestamp strings
ist_start = datetime.strptime(ist_start_str, "%Y-%m-%d %H:%M:%S.%f")
ist_end   = datetime.strptime(ist_end_str,   "%Y-%m-%d %H:%M:%S.%f")

# Convert IST -> UTC
utc_start = ist_start - timedelta(hours=5, minutes=30)
utc_end   = ist_end   - timedelta(hours=5, minutes=30)


# EXTRACTION_DIR should be a pathlib.Path
conn, sql_query = setup_duckdb_query(
    output_dir=EXTRACTION_DIR,
    utc_start=utc_start,
    utc_end=utc_end,
    core_cols=None,   # <-- ALL COLUMNS
    ids=ids,          # <-- Filter by IDs
)

df_raw = conn.execute(sql_query).fetchdf()
# Compute the correct position
ts_index = df_raw.columns.get_loc("timestamp")

# Insert at index+1
df_raw.insert(loc=ts_index + 1, column="timestamp_ist",
          value=(df_raw["timestamp"] + pd.Timedelta(hours=5, minutes=30))
                 .dt.strftime("%Y-%m-%d %H:%M:%S.%f"))        
df_raw["timestamp_ist"] = pd.to_datetime(df_raw["timestamp_ist"], format="%Y-%m-%d %H:%M:%S.%f")

In [10]:
df_raw.head().to_csv("df_raw_head.csv", index=False)

In [11]:
(df_raw.isna().sum()*100/len(df_raw)).round(2).to_csv("df_raw_na_counts.csv", index=True)

In [12]:
df_raw[df_raw.pack_temperature1.notna()].head()
# df_raw.head()

,id,timestamp,timestamp_ist,insert_timestamp,sequence,number_of_can_ids,number_of_records,percentage_of_can_ids,lowpressureoilpumpfaultcode,lowvoltagedcaclifesignal,batterycoolingstate,batterycoolanttemperature,odometerreading,brakepedalpos,ebsredwarningsignal,abs_ebsamberwarningsignal,insulation_status,insulation__value,lifesignalofinsulation,output_phase_voltage,output_phase_currant,motor_rpm,motor_torque,motor_temperature,igbt_temperature,mcuself_teststatus,side_door_panel_open_alarm,ignitionstatus,brakingsystemmallfunction_alarm,parking_brake_status,front_air_pressure,rear_air_pressure,brake_pad_worn_out_alarm,guna_dcp_temperature,guna_dcm_temperature,gunb_dcp_temperature,gunb_dcm_temperature,gunc_dcp_temperature,gunc_dcm_temperature,batterypackovervoltagealarm,monomerovervoltagealarm,polehightemperaturealarm,batteryhightemperaturealarm,hightempalarmofcharginggun,monomerundervoltagealarm,batterypackundervoltagealarm,temperaturedifferencealarm,singlevoltagedifferencealarm,chargingcurrentalarm,dischargecurrentalarm,sochighalarm,soclowalarm,bms_fault_code,gun_connection_status,maxavailableshorttermcharging,maxavailableshorttimedischarge,chargingcontactor2positive,chargingcontactor2negative,chargingcontactor1positive,batterytotalnegativecontactor,chargingcontactor1negative,pack_negative_contactors_status,bat_soc,bat_voltage,total_battery_current,bmslifesignal,steering_pump_enable_command,air_compressor_enable_command,dcdc_enable_command,vcuversioninformation,distancetoempty,accelarationpedal,brake_pedal,vcuself_teststatus,systeminterlockstate,vehiclereadycondition,vehicle_speed_vcu,gear_position,autoholdunenable,powertrain_failure_alarm,vcu_fault_code,vehiclecontrollerlife,cellmax_voltagecellnumber,cell_max_voltage,cellminvoltagecellnumber,cell_min_voltage,pack1_cellmax_temperature,pack1_maxtemperature_cell_number,pack1_cell_min_temperature,pack1_celltemperature_cellnumber,fourinonelifesignal,fourinoneself_teststatus,prechargedcontactorstate,mainmotorcontactor1state,accessorycontactorstatus,pre_charge_status,fiveinone_faultcode,airpumpcondition,air_compreesor_temperature,airpumpdcaclifesignal,dcdc_statusandfailure,dcdc_out_put_currant,dcdc_out_put_volatge,dcdcbusvoltage,dcdclifesignal,oilpumpcondition,steering_pump_rpm,steeringcontroller_failurealarm,lift_lower_normal_request,kneeling_request,th_management_sys_working_mode,soh,charging_contactor_3_positive,charging_contactor_3_negative,charger_voltage,charger_current,chrargecurrent_request,evcc_error,chargingerror_bms,vehicle_operation_mode,pack_temperature1,pack_temperature2,pack_temperature3,pack_temperature4,pack_temperature5,pack_temperature6,pack_temperature7,pack_temperature8,pack_temperature9,pack_temperature10,pack_temperature11,pack_temperature12,pack_temperature13,pack_temperature14,pack_temperature15,pack_temperature16,pack_temperature17,pack_temperature18,pack_temperature19,pack_temperature20,pack_temperature21,pack_temperature22,pack_temperature23,pack_temperature24,pack_temperature25,pack_temperature26,pack_temperature27,pack_temperature28,pack_temperature29,pack_temperature30,pack_temperature31,pack_temperature32,pack_temperature33,pack_temperature34,pack_temperature35,pack_temperature36,pack_temperature37,pack_temperature38,pack_temperature39,pack_temperature40,pack_temperature41,pack_temperature42,pack_temperature43,pack_temperature44,pack_temperature45,pack_temperature46,pack_temperature47,pack_temperature48,pack_temperature49,pack_temperature50,pack_temperature51,pack_temperature52,pack_temperature53,pack_temperature54,pack_temperature55,pack_temperature56,pack_temperature57,pack_temperature58,pack_temperature59,pack_temperature60,pack_temperature61,pack_temperature62,pack_temperature63,pack_temperature64,pack_temperature65,pack_temperature66,pack_temperature67,pack_temperature68,pack_temperature69,pack_temperature70,pack_temperature71,pack_temperature72,pack_temperature73,pack_temperature74,pack_temperature75,pack_temperature76,pack_temperature77,pack_tempe

In [13]:
volt_cols = [c for c in df_raw.columns if c.startswith("pack_cellvoltage_")]
len(volt_cols)

576

In [14]:
# df_raw.isnull().sum()*100/len(df_raw)

In [15]:
def extract_voltage_sweeps(df, volt_cols, timestamp_col="timestamp_ist"):
    """
    Detects complete voltage sweeps (ignoring cells 81–84).
    Returns: cycle_start, cycle_end, duration_sec, interval_sec,
             + voltage values for all valid 572 cells.
    """

    # -------- STEP 1 — Identify valid columns (exclude 81–84) --------
    def get_cell_num(col):
        return int(col.split("_")[-1])

    ignore_cells = {81, 82, 83, 84}

    valid_volt_cols = [c for c in volt_cols if get_cell_num(c) not in ignore_cells]
    valid_cell_indices = [get_cell_num(c) for c in valid_volt_cols]

    EXPECTED = len(valid_cell_indices)  # should be 572
    print(f"Using {EXPECTED} expected cells (ignoring 81–84).")

    # -------- STEP 2 — Sort & prepare dataframe --------
    df = df.sort_values(timestamp_col).reset_index(drop=True)

    # For each row, identify which cells appear
    df["cells_present"] = df[valid_volt_cols].notna().apply(
        lambda row: [valid_cell_indices[i] for i, ok in enumerate(row) if ok],
        axis=1
    )

    # -------- STEP 3 — Sweep detection logic --------
    sweeps = []
    seen = set()
    cycle_start = None

    for _, row in df.iterrows():
        ts = row[timestamp_col]
        cells = row["cells_present"]

        if len(seen) == 0:
            cycle_start = ts

        seen.update(cells)

        # Full sweep detected (572 cells)
        if len(seen) == EXPECTED:
            sweeps.append({"cycle_start": cycle_start, "cycle_end": ts})
            seen = set()

    # -------- STEP 4 — No sweeps found --------
    if len(sweeps) == 0:
        print("⚠ No complete sweeps detected for 572 valid cells.")
        return pd.DataFrame(columns=["cycle_start","cycle_end","duration_sec","interval_sec"] + valid_volt_cols)

    # -------- STEP 5 — Build sweep dataframe --------
    df_cycles = pd.DataFrame(sweeps)
    df_cycles["duration_sec"] = (df_cycles["cycle_end"] - df_cycles["cycle_start"]).dt.total_seconds()
    df_cycles["interval_sec"] = df_cycles["cycle_end"].diff().dt.total_seconds()
    df_cycles.loc[0, "interval_sec"] = None

    # -------- STEP 6 — Extract voltage values for each sweep --------
    output_rows = []

    for _, cyc in df_cycles.iterrows():
        start, end = cyc["cycle_start"], cyc["cycle_end"]

        # Filter for rows in this sweep
        mask = (df[timestamp_col] >= start) & (df[timestamp_col] <= end)
        subset = df.loc[mask]

        if subset.empty:
            print(f"⚠ Warning: empty subset for sweep starting {start}. Skipping.")
            continue

        # Forward/backfill to get final voltage values
        valid_vals = subset[valid_volt_cols].ffill().bfill()
        if valid_vals.empty:
            print(f"⚠ Warning: no valid volt data for sweep starting {start}. Skipping.")
            continue

        volt_values = valid_vals.iloc[-1]  # final values

        # Build output row
        row_out = {
            "cycle_start": start,
            "cycle_end": end,
            "duration_sec": cyc["duration_sec"],
            "interval_sec": cyc["interval_sec"]
        }

        for col in valid_volt_cols:
            row_out[col] = volt_values[col]

        output_rows.append(row_out)

    return pd.DataFrame(output_rows)


volt_cols = [c for c in df_raw.columns if c.startswith("pack_cellvoltage_")]

df_sweeps_572 = extract_voltage_sweeps(df_raw, volt_cols)
df_sweeps_572


Using 572 expected cells (ignoring 81–84).


,cycle_start,cycle_end,duration_sec,interval_sec,pack_cellvoltage_1,pack_cellvoltage_2,pack_cellvoltage_3,pack_cellvoltage_4,pack_cellvoltage_5,pack_cellvoltage_6,pack_cellvoltage_7,pack_cellvoltage_8,pack_cellvoltage_9,pack_cellvoltage_10,pack_cellvoltage_11,pack_cellvoltage_12,pack_cellvoltage_13,pack_cellvoltage_14,pack_cellvoltage_15,pack_cellvoltage_16,pack_cellvoltage_17,pack_cellvoltage_18,pack_cellvoltage_19,pack_cellvoltage_20,pack_cellvoltage_21,pack_cellvoltage_22,pack_cellvoltage_23,pack_cellvoltage_24,pack_cellvoltage_25,pack_cellvoltage_26,pack_cellvoltage_27,pack_cellvoltage_28,pack_cellvoltage_29,pack_cellvoltage_30,pack_cellvoltage_31,pack_cellvoltage_32,pack_cellvoltage_33,pack_cellvoltage_34,pack_cellvoltage_35,pack_cellvoltage_36,pack_cellvoltage_37,pack_cellvoltage_38,pack_cellvoltage_39,pack_cellvoltage_40,pack_cellvoltage_41,pack_cellvoltage_42,pack_cellvoltage_43,pack_cellvoltage_44,pack_cellvoltage_45,pack_cellvoltage_46,pack_cellvoltage_47,pack_cellvoltage_48,pack_cellvoltage_49,pack_cellvoltage_50,pack_cellvoltage_51,pack_cellvoltage_52,pack_cellvoltage_53,pack_cellvoltage_54,pack_cellvoltage_55,pack_cellvoltage_56,pack_cellvoltage_57,pack_cellvoltage_58,pack_cellvoltage_59,pack_cellvoltage_60,pack_cellvoltage_61,pack_cellvoltage_62,pack_cellvoltage_63,pack_cellvoltage_64,pack_cellvoltage_65,pack_cellvoltage_66,pack_cellvoltage_67,pack_cellvoltage_68,pack_cellvoltage_69,pack_cellvoltage_70,pack_cellvoltage_71,pack_cellvoltage_72,pack_cellvoltage_73,pack_cellvoltage_74,pack_cellvoltage_75,pack_cellvoltage_76,pack_cellvoltage_77,pack_cellvoltage_78,pack_cellvoltage_79,pack_cellvoltage_80,pack_cellvoltage_85,pack_cellvoltage_86,pack_cellvoltage_87,pack_cellvoltage_88,pack_cellvoltage_89,pack_cellvoltage_90,pack_cellvoltage_91,pack_cellvoltage_92,pack_cellvoltage_93,pack_cellvoltage_94,pack_cellvoltage_95,pack_cellvoltage_96,pack_cellvoltage_97,pack_cellvoltage_98,pack_cellvoltage_99,pack_cellvoltage_100,pack_cellvoltage_101,pack_cellvoltage_102,pack_cellvoltage_103,pack_cellvoltage_104,pack_cellvoltage_105,pack_cellvoltage_106,pack_cellvoltage_107,pack_cellvoltage_108,pack_cellvoltage_109,pack_cellvoltage_110,pack_cellvoltage_111,pack_cellvoltage_112,pack_cellvoltage_113,pack_cellvoltage_114,pack_cellvoltage_115,pack_cellvoltage_116,pack_cellvoltage_117,pack_cellvoltage_118,pack_cellvoltage_119,pack_cellvoltage_120,pack_cellvoltage_121,pack_cellvoltage_122,pack_cellvoltage_123,pack_cellvoltage_124,pack_cellvoltage_125,pack_cellvoltage_126,pack_cellvoltage_127,pack_cellvoltage_128,pack_cellvoltage_129,pack_cellvoltage_130,pack_cellvoltage_131,pack_cellvoltage_132,pack_cellvoltage_133,pack_cellvoltage_134,pack_cellvoltage_135,pack_cellvoltage_136,pack_cellvoltage_137,pack_cellvoltage_138,pack_cellvoltage_139,pack_cellvoltage_140,pack_cellvoltage_141,pack_cellvoltage_142,pack_cellvoltage_143,pack_cellvoltage_144,pack_cellvoltage_145,pack_cellvoltage_146,pack_cellvoltage_147,pack_cellvoltage_148,pack_cellvoltage_149,pack_cellvoltage_150,pack_cellvoltage_151,pack_cellvoltage_152,pack_cellvoltage_153,pack_cellvoltage_154,pack_cellvoltage_155,pack_cellvoltage_156,pack_cellvoltage_157,pack_cellvoltage_158,pack_cellvoltage_159,pack_cellvoltage_160,pack_cellvoltage_161,pack_cellvoltage_162,pack_cellvoltage_163,pack_cellvoltage_164,pack_cellvoltage_165,pack_cellvoltage_166,pack_cellvoltage_167,pack_cellvoltage_168,pack_cellvoltage_169,pack_cellvoltage_170,pack_cellvoltage_171,pack_cellvoltage_172,pack_cellvoltage_173,pack_cellvoltage_174,pack_cellvoltage_175,pack_cellvoltage_176,pack_cellvoltage_177,pack_cellvoltage_178,pack_cellvoltage_179,pack_cellvoltage_180,pack_cellvoltage_181,pack_cellvoltage_182,pack_cellvoltage_183,pack_cellvoltage_184,pack_cellvoltage_185,pack_cellvoltage_186,pack_cellvoltage_187,pack_cellvoltage_188,pack_cellvoltage_189,pack_cellvoltage_190,pack_cellvoltage_191,pack_cellvoltage_192,pack_cellvoltage_193,pack_cellvoltage_194,pack_cellvoltage_195,pack_cellvoltage_196,pack_cellvoltage_197,pa

In [16]:
df_raw.id.value_counts()

id
28    544
Name: count, dtype: int64

In [17]:
import pandas as pd

# Load your file
df = pd.read_csv("bb_id28_6084_271125_1300hrs_10mins.csv")
display(df.head())
# If your columns have duplicate names like timestamp,timestamp,
# pandas will automatically rename them timestamp and timestamp.1
# Let's identify the actual name of the second timestamp column:
ts_cols = [c for c in df.columns if c.startswith("timestamp")]

# Keep only the FIRST timestamp column
timestamp_col = ts_cols[0]

# Build the desired column order
byte_cols = [f"byte{i}" for i in range(1, 9)]

new_order = [
    "id",
    timestamp_col,
    "sequence",
    "can_id",
] + byte_cols

# Reorder the dataframe
df_reordered = df[new_order]

# Save back to CSV
df_reordered.to_csv("bb_id28_6084_271125_1300hrs_10mins_reordered.csv", index=False)
df_reordered.head()


,id,timestamp,timestamp.1,sequence,byte1,byte8,byte6,byte3,can_id,byte4,byte2,byte5,byte7
0,28,1764229199971,1764229199971,39680,255,255,255,107,419343920,107,255,255,255
1,28,1764229199971,1764229199971,39702,0,19,0,0,418710512,0,0,0,136
2,28,1764229199971,1764229199971,39682,14,190,255,0,217449893,74,1,0,0
3,28,1764229199971,1764229199971,39683,0,0,0,0,150892043,0,0,0,0
4,28,1764229199971,1764229199971,39684,0,255,255,255,419354928,255,255,255,255


,id,timestamp,sequence,can_id,byte1,byte2,byte3,byte4,byte5,byte6,byte7,byte8
0,28,1764229199971,39680,419343920,255,255,107,107,255,255,255,255
1,28,1764229199971,39702,418710512,0,0,0,0,0,0,136,19
2,28,1764229199971,39682,217449893,14,1,0,74,0,255,0,190
3,28,1764229199971,39683,150892043,0,0,0,0,0,0,0,0
4,28,1764229199971,39684,419354928,0,255,255,255,255,255,255,255


In [18]:
df_reordered.head().to_csv("df_reordered_head.csv", index=False)

In [19]:
# df_reordered["can_id_adj"] = drop_msb_and_convert(df_reordered["can_id"].tolist())

In [20]:
def drop_msb(can_id):
    """
    Removes the 31st bit from the DBC CAN ID.
    Produces the raw CAN ID observed in your logs.
    """
    return can_id & 0x7FFFFFFF


In [21]:
dbc_lines = open("canId_conversions.txt").read().splitlines()

dbc_ids = []

for line in dbc_lines:
    if line.startswith("BO_ "):
        parts = line.split()
        dbc_id = int(parts[1])
        dbc_ids.append(dbc_id)

dbc_ids = sorted(set(dbc_ids))
len(dbc_ids)
raw_ids = [drop_msb(cid) for cid in dbc_ids]


In [22]:
raw_ids = [drop_msb(cid) for cid in dbc_ids]

df_map = pd.DataFrame({
    "dbc_can_id": dbc_ids,
    "raw_can_id": raw_ids,
})

df_map["dbc_hex"] = df_map["dbc_can_id"].apply(lambda x: hex(x))
df_map["raw_hex"] = df_map["raw_can_id"].apply(lambda x: hex(x))

df_map.to_csv("dbc_to_raw_can_id_mapping.csv", index=False)


In [23]:
df_map = df_map.rename(columns={
    "dbc_can_id": "dbc_id",
    "raw_can_id": "raw_id"
})
df_map.head()

,dbc_id,raw_id,dbc_hex,raw_hex
0,2556368883,408885235,0x985f17f3,0x185f17f3
1,2556434419,408950771,0x986017f3,0x186017f3
2,2556499955,409016307,0x986117f3,0x186117f3
3,2556565491,409081843,0x986217f3,0x186217f3
4,2556631027,409147379,0x986317f3,0x186317f3


In [24]:
# RAW → DBC (for decoding incoming CAN messages)
raw_to_dbc = dict(zip(df_map["raw_id"], df_map["dbc_id"]))

# DBC → RAW (if you ever need it)
dbc_to_raw = dict(zip(df_map["dbc_id"], df_map["raw_id"]))


In [25]:
df_reordered["dbc_id"] = df_reordered["can_id"].map(raw_to_dbc)
df_reordered.dbc_id = df_reordered.dbc_id.astype("Int64")
df_reordered[~df_reordered["dbc_id"].isna()].head()

,id,timestamp,sequence,can_id,byte1,byte2,byte3,byte4,byte5,byte6,byte7,byte8,dbc_id
79,28,1764229199851,39620,408885235,108,9,74,8,72,27,255,255,2556368883
82,28,1764229199851,39623,410982387,216,1,58,13,193,0,8,13,2558466035
95,28,1764229199851,39636,418518259,72,72,71,72,71,71,0,0,2566001907
106,28,1764229199831,39585,411047923,9,13,12,13,10,13,10,13,2558531571
109,28,1764229199831,39588,408950771,73,74,74,74,74,73,73,74,2556434419


In [26]:
df_reordered.dbc_id.sort_values().unique()

<IntegerArray>
[2556368883, 2556434419, 2556499955, 2556565491, 2556631027, 2556696563,
 2556762099, 2556827635, 2556893171, 2556958707,
 ...
 2567378931, 2567444467, 2567510003, 2567575539, 2567641075, 2567706611,
 2567772147, 2567837683, 2567903219,       <NA>]
Length: 158, dtype: Int64

In [27]:
decoded = df_reordered[["id","timestamp","sequence"]].copy()

# Empty columns
for i in range(1, 577):
    decoded[f"Pack_cellVoltage_{i}"] = None

for i in range(1, 109):
    decoded[f"Pack_Temperature_{i}"] = None

byte_cols = [f"byte{i}" for i in range(1,9)]
df_reordered["payload"] = df_reordered[byte_cols].values.tolist()
display(df_reordered.head())
display(decoded[decoded.Pack_cellVoltage_1.notnull()].head())

,id,timestamp,sequence,can_id,byte1,byte2,byte3,byte4,byte5,byte6,byte7,byte8,dbc_id,payload
0,28,1764229199971,39680,419343920,255,255,107,107,255,255,255,255,<NA>,"[255, 255, 107, 107, 255, 255, 255, 255]"
1,28,1764229199971,39702,418710512,0,0,0,0,0,0,136,19,<NA>,"[0, 0, 0, 0, 0, 0, 136, 19]"
2,28,1764229199971,39682,217449893,14,1,0,74,0,255,0,190,<NA>,"[14, 1, 0, 74, 0, 255, 0, 190]"
3,28,1764229199971,39683,150892043,0,0,0,0,0,0,0,0,<NA>,"[0, 0, 0, 0, 0, 0, 0, 0]"
4,28,1764229199971,39684,419354928,0,255,255,255,255,255,255,255,<NA>,"[0, 255, 255, 255, 255, 255, 255, 255]"


,id,timestamp,sequence,Pack_cellVoltage_1,Pack_cellVoltage_2,Pack_cellVoltage_3,Pack_cellVoltage_4,Pack_cellVoltage_5,Pack_cellVoltage_6,Pack_cellVoltage_7,Pack_cellVoltage_8,Pack_cellVoltage_9,Pack_cellVoltage_10,Pack_cellVoltage_11,Pack_cellVoltage_12,Pack_cellVoltage_13,Pack_cellVoltage_14,Pack_cellVoltage_15,Pack_cellVoltage_16,Pack_cellVoltage_17,Pack_cellVoltage_18,Pack_cellVoltage_19,Pack_cellVoltage_20,Pack_cellVoltage_21,Pack_cellVoltage_22,Pack_cellVoltage_23,Pack_cellVoltage_24,Pack_cellVoltage_25,Pack_cellVoltage_26,Pack_cellVoltage_27,Pack_cellVoltage_28,Pack_cellVoltage_29,Pack_cellVoltage_30,Pack_cellVoltage_31,Pack_cellVoltage_32,Pack_cellVoltage_33,Pack_cellVoltage_34,Pack_cellVoltage_35,Pack_cellVoltage_36,Pack_cellVoltage_37,Pack_cellVoltage_38,Pack_cellVoltage_39,Pack_cellVoltage_40,Pack_cellVoltage_41,Pack_cellVoltage_42,Pack_cellVoltage_43,Pack_cellVoltage_44,Pack_cellVoltage_45,Pack_cellVoltage_46,Pack_cellVoltage_47,Pack_cellVoltage_48,Pack_cellVoltage_49,Pack_cellVoltage_50,Pack_cellVoltage_51,Pack_cellVoltage_52,Pack_cellVoltage_53,Pack_cellVoltage_54,Pack_cellVoltage_55,Pack_cellVoltage_56,Pack_cellVoltage_57,Pack_cellVoltage_58,Pack_cellVoltage_59,Pack_cellVoltage_60,Pack_cellVoltage_61,Pack_cellVoltage_62,Pack_cellVoltage_63,Pack_cellVoltage_64,Pack_cellVoltage_65,Pack_cellVoltage_66,Pack_cellVoltage_67,Pack_cellVoltage_68,Pack_cellVoltage_69,Pack_cellVoltage_70,Pack_cellVoltage_71,Pack_cellVoltage_72,Pack_cellVoltage_73,Pack_cellVoltage_74,Pack_cellVoltage_75,Pack_cellVoltage_76,Pack_cellVoltage_77,Pack_cellVoltage_78,Pack_cellVoltage_79,Pack_cellVoltage_80,Pack_cellVoltage_81,Pack_cellVoltage_82,Pack_cellVoltage_83,Pack_cellVoltage_84,Pack_cellVoltage_85,Pack_cellVoltage_86,Pack_cellVoltage_87,Pack_cellVoltage_88,Pack_cellVoltage_89,Pack_cellVoltage_90,Pack_cellVoltage_91,Pack_cellVoltage_92,Pack_cellVoltage_93,Pack_cellVoltage_94,Pack_cellVoltage_95,Pack_cellVoltage_96,Pack_cellVoltage_97,Pack_cellVoltage_98,Pack_cellVoltage_99,Pack_cellVoltage_100,Pack_cellVoltage_101,Pack_cellVoltage_102,Pack_cellVoltage_103,Pack_cellVoltage_104,Pack_cellVoltage_105,Pack_cellVoltage_106,Pack_cellVoltage_107,Pack_cellVoltage_108,Pack_cellVoltage_109,Pack_cellVoltage_110,Pack_cellVoltage_111,Pack_cellVoltage_112,Pack_cellVoltage_113,Pack_cellVoltage_114,Pack_cellVoltage_115,Pack_cellVoltage_116,Pack_cellVoltage_117,Pack_cellVoltage_118,Pack_cellVoltage_119,Pack_cellVoltage_120,Pack_cellVoltage_121,Pack_cellVoltage_122,Pack_cellVoltage_123,Pack_cellVoltage_124,Pack_cellVoltage_125,Pack_cellVoltage_126,Pack_cellVoltage_127,Pack_cellVoltage_128,Pack_cellVoltage_129,Pack_cellVoltage_130,Pack_cellVoltage_131,Pack_cellVoltage_132,Pack_cellVoltage_133,Pack_cellVoltage_134,Pack_cellVoltage_135,Pack_cellVoltage_136,Pack_cellVoltage_137,Pack_cellVoltage_138,Pack_cellVoltage_139,Pack_cellVoltage_140,Pack_cellVoltage_141,Pack_cellVoltage_142,Pack_cellVoltage_143,Pack_cellVoltage_144,Pack_cellVoltage_145,Pack_cellVoltage_146,Pack_cellVoltage_147,Pack_cellVoltage_148,Pack_cellVoltage_149,Pack_cellVoltage_150,Pack_cellVoltage_151,Pack_cellVoltage_152,Pack_cellVoltage_153,Pack_cellVoltage_154,Pack_cellVoltage_155,Pack_cellVoltage_156,Pack_cellVoltage_157,Pack_cellVoltage_158,Pack_cellVoltage_159,Pack_cellVoltage_160,Pack_cellVoltage_161,Pack_cellVoltage_162,Pack_cellVoltage_163,Pack_cellVoltage_164,Pack_cellVoltage_165,Pack_cellVoltage_166,Pack_cellVoltage_167,Pack_cellVoltage_168,Pack_cellVoltage_169,Pack_cellVoltage_170,Pack_cellVoltage_171,Pack_cellVoltage_172,Pack_cellVoltage_173,Pack_cellVoltage_174,Pack_cellVoltage_175,Pack_cellVoltage_176,Pack_cellVoltage_177,Pack_cellVoltage_178,Pack_cellVoltage_179,Pack_cellVoltage_180,Pack_cellVoltage_181,Pack_cellVoltage_182,Pack_cellVoltage_183,Pack_cellVoltage_184,Pack_cellVoltage_185,Pack_cellVoltage_186,Pack_cellVoltage_187,Pack_cellVoltage_188,Pack_cellVoltage_189,Pack_cellVoltage_190,Pack_cellVoltage_191,Pack_cellVoltage_192,Pack_cellVoltage_193,Pack_cellVoltage_194,Pack_cellVo

In [28]:
for idx, row in df_reordered.iterrows():
    dbc = row["dbc_id"]
    bytes_ = row["payload"]
    
    # Temperature frames
    if dbc in temp_map:
        sensor_indices = temp_map[dbc]
        for k, sensor_id in enumerate(sensor_indices):
            if sensor_id <= 108:
                decoded.loc[idx, f"Pack_Temperature_{sensor_id}"] = bytes_[k] - 40

    # Voltage frames
    if dbc in volt_map:
        cells = volt_map[dbc]
        for k, cell_index in enumerate(cells):
            L = bytes_[k*2]       # lower byte
            H = bytes_[k*2 + 1]   # higher byte

            raw_val = (H << 8) | L
            decoded_value = raw_val * 0.001  # as per DBC scaling

            decoded.loc[idx, f"Pack_cellVoltage_{cell_index}"] = decoded_value


In [29]:
decoded.sort_values(by="timestamp", inplace=True)

In [30]:
# Compute the correct position
ts_index = decoded.columns.get_loc("timestamp")

# Insert at index+1
decoded.insert(loc=ts_index + 1, column="timestamp_ist",
          value=(pd.to_datetime(decoded["timestamp"], unit="ms", utc=True).dt.tz_convert("Asia/Kolkata").dt.tz_localize(None)))

# decoded["timestamp_ist"] = pd.to_datetime(decoded["timestamp_ist"], format="%Y-%m-%d %H:%M:%S.%f")

In [31]:
# decoded[decoded.Pack_cellVoltage_1.notna()|decoded.Pack_cellVoltage_5.notna()|decoded.Pack_cellVoltage_9.notna()|decoded.Pack_cellVoltage_13.notna()|decoded.Pack_cellVoltage_17.notna()|decoded.Pack_cellVoltage_21.notna()|decoded.Pack_cellVoltage_25.notna()]

In [32]:
# Get all cell voltage columns
volt_cols = [c for c in decoded.columns if c.startswith("Pack_cellVoltage_")]

# Extract numeric index from each column
cell_indices = [int(c.split("_")[-1]) for c in volt_cols]
num_cells = len(cell_indices)  # should be 576

decoded["cells_present"] = decoded[volt_cols].notna().apply(lambda row: 
    [cell_indices[i] for i, present in enumerate(row) if present], axis=1)

cycles = []
current_cells = set()
cycle_start_time = None

for idx, row in decoded.iterrows():
    cells = row["cells_present"]
    ts = row["timestamp_ist"]

    # Start of a new cycle
    if len(current_cells) == 0:
        cycle_start_time = ts

    # Add newly seen cells
    current_cells.update(cells)

    # If full cycle achieved
    if len(current_cells) == num_cells:
        cycle_end_time = ts
        duration = (cycle_end_time - cycle_start_time).total_seconds()

        cycles.append({
            "cycle_start": cycle_start_time,
            "cycle_end": cycle_end_time,
            "duration_sec": duration
        })

        # Reset for next cycle
        current_cells = set()

df_cycles = pd.DataFrame(cycles)
display(df_cycles)


,cycle_start,cycle_end,duration_sec
0,2025-11-27 13:00:20.818,2025-11-27 13:02:23.251,122.433
1,2025-11-27 13:02:23.671,2025-11-27 13:05:00.791,157.120
2,2025-11-27 13:05:01.031,2025-11-27 13:06:24.831,83.800
3,2025-11-27 13:06:25.011,2025-11-27 13:08:00.930,95.919
4,2025-11-27 13:08:01.131,2025-11-27 13:08:50.191,49.060
5,2025-11-27 13:08:50.371,2025-11-27 13:09:45.831,55.460


In [ ]:
# volt_cols = [c for c in decoded.columns if c.startswith("Pack_cellVoltage_")]
# cell_indices = [int(c.split("_")[-1]) for c in volt_cols]

def extract_cycle_cell_values(decoded, volt_cols):
    """
    Detects full 576-cell cycles and extracts:
      - cycle_start
      - cycle_end
      - duration_sec
      - interval_sec
      - all 576 Pack_cellVoltage columns

    Returns:
        DataFrame with one row per complete cycle
    """

    cell_indices = [int(c.split("_")[-1]) for c in volt_cols]
    num_cells = len(cell_indices)

    cycles = []
    current_cells = set()
    cycle_start_time = None

    # STEP 1 — DETECT CYCLES
    for idx, row in decoded.iterrows():
        cells_present = [
            cell_indices[i] for i, present in enumerate(row[volt_cols].notna()) if present
        ]
        ts = row["timestamp_ist"]

        # Start of new sweep
        if len(current_cells) == 0:
            cycle_start_time = ts

        # Add cells detected in this row
        current_cells.update(cells_present)

        # Full sweep detected
        if len(current_cells) == num_cells:
            cycle_end_time = ts

            cycles.append({
                "cycle_start": cycle_start_time,
                "cycle_end": cycle_end_time
            })

            # Reset for next cycle
            current_cells = set()

    # STEP 2 — CALCULATE DURATION + INTERVAL
    df_cycles = pd.DataFrame(cycles)
    df_cycles["duration_sec"] = (df_cycles["cycle_end"] - df_cycles["cycle_start"]).dt.total_seconds()
    df_cycles["interval_sec"] = df_cycles["cycle_end"].diff().dt.total_seconds()
    df_cycles.loc[0, "interval_sec"] = None  # first cycle has no previous interval

    # STEP 3 — EXTRACT 576 CELL VALUES FOR EACH CYCLE
    result_rows = []

    for i, row in df_cycles.iterrows():
        start, end = row["cycle_start"], row["cycle_end"]

        # Filter data inside this cycle interval
        mask = (decoded["timestamp_ist"] >= start) & (decoded["timestamp_ist"] <= end)
        segment = decoded.loc[mask, volt_cols]

        # Forward/backfill and take the final available value
        final_values = segment.ffill().bfill().iloc[-1]

        # Build output row
        out = {
            "cycle_start": start,
            "cycle_end": end,
            "duration_sec": row["duration_sec"],
            "interval_sec": row["interval_sec"],
        }

        # Append all 576 cell values
        for col in volt_cols:
            out[col] = final_values[col]

        result_rows.append(out)

    return pd.DataFrame(result_rows)

volt_cols = [c for c in decoded.columns if c.startswith("Pack_cellVoltage_")]

df_cycle_voltages = extract_cycle_cell_values(decoded, volt_cols)

display(df_cycle_voltages)

In [ ]:
df_cycle_voltages.cycle_start.min(),df_cycle_voltages.cycle_end.max()